# Computational Cognitive Neuroscience

Douwe van Erp (s4258126) & Arianne Meijer - van de Griend (s4620135)

# Assignment 1 - Training an MLP on MNIST
 
The goal of the practical assignments is to become proficient at the development of SOTA neural network models. In this first assignment, you will learn to implement a multilayer perceptron in [Chainer](https://chainer.org). If you have little prior experience with neural networks, you should acquaint yourself with the basics. To this end you should read at least Chapters 1 and 2 on the following [website](http://neuralnetworksanddeeplearning.com).
 
To prepare yourself, you should study the [Chainer tutorial](https://docs.chainer.org/en/stable/tutorial/index.html). To really understand the framework, you are required to use low-level Chainer code. This means that you cannot use the Trainer and Updater objects (that is, you need to implement the training loops yourself). Note also that it is easiest to implement your own data iterators. We provide an iterator which creates batches of data in utils.py. It can be convenient and is allowed to use the Classifier object.

## Instructions
Train a multilayer perceptron which has one hidden layer consisting of 10 hidden units on the MNIST dataset. The goal is to classify handwritten characters. To speed up training, use get_mnist in utils.py with n_train=n_test=100. Train the network for 20 training epochs. Use a batch size of 32. Plot the decrease in training and test loss over epochs. Submit your code as a documented Jupyter notebook, which includes your figure. For development, it is advised to first implement your work in PyCharm and then port your code to a notebook.


In [1]:
import chainer.functions as F
import chainer.links as L
from chainer import Chain
from chainer import iterators, optimizers
from chainer import report, training
from chainer.training import extensions
import utils

Implementation of the multilayer perceptron class

In [2]:
class MLP(Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        y = self.l2(h1)
        return y

Implementation of a Classifier with softmax cross entropy as loss function

In [3]:
class Classifier(Chain):
    def __init__(self,predictor):
        super(Classifier, self).__init__()
        self.predictor = predictor
        
    def __call__(self, x, t):
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        report({'loss': loss, 'accuracy' : accuracy}, self)
        return loss

Get the train and test data and define the iterators for those datasets.

In [4]:
train, test = utils.get_mnist(n_train=100, n_test=100, n_dim=1, with_label=True)

# Batch size 32
train_iter = iterators.SerialIterator(train, batch_size=32, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size=32, repeat=False, shuffle=False)

Chain the different components of the model.
Define the model, optimizer, updater and trainer.
Also add the visualizations of the progress to the trainer.
Then run the trainer.

In [5]:
model = L.Classifier(MLP(10, 10))  # the input size, 784, is inferred
optimizer = optimizers.SGD()
optimizer.setup(model)

updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out='result')
trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.LogReport())
#trainer.extend(extensions.PlotReport(['main/accuracy'],'iteration'))
#trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], 'epoch'))
trainer.extend(extensions.ProgressBar())
trainer.run()

     total [########..........................................] 16.00%
this epoch [##########........................................] 20.00%
       100 iter, 3 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [################..................................] 32.00%
this epoch [####################..............................] 40.00%
       200 iter, 6 epoch / 20 epochs
     122.1 iters/sec. Estimated time to finish: 0:00:03.480750.
     total [########################..........................] 48.00%
this epoch [#############################.....................] 60.00%
       300 iter, 9 epoch / 20 epochs
    123.92 iters/sec. Estimated time to finish: 0:00:02.622750.
     total [################################..................] 64.00%
this epoch [########################################..........] 80.00%
       400 iter, 12 epoch / 20 epochs
    126.85 iters/sec. Estimated time to finish: 0:00:01.773750.
     total [######################

![](result/plot.png)